# Create a Perturbation in a Material

Create a perturbation in a material with a specified smooth function (SineWave by default)

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Notes

1. For more information, see [Introduction](Introduction.ipynb)


## 1. Prepare the Environment
### 1.1. Set up perturbation parameters

In [ ]:
# Sine wave perturbation parameters
AMPLITUDE = 0.5
WAVELENGTH = 1.0
PHASE = 0.0
AXIS = "y"

USE_CARTESIAN_COORDINATES = False

SUPERCELL_MATRIX = [[30, 0, 0], [0, 30, 0], [0, 0, 1]]

PRESERVE_GEODESIC_DISTANCE = False

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("")

### 1.3. Get input materials
Materials are loaded with `get_materials()`.

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

### 1.4. Create and preview Supercell

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.supercell import create_supercell

unit_cell = materials[0]
supercell = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create the SineWave Perturbation
### 2.1. Set sine wave perturbation parameters
Use perturbation function imported from `mat3ra.made.tools.utils.perturbation` folder.

In [ ]:
from mat3ra.made.tools.build.perturbation import PerturbationConfiguration, SlabPerturbationBuilder, CellMatchingDistancePreservingSlabPerturbationBuilder
from mat3ra.made.tools.utils.perturbation import SineWavePerturbationFunctionHolder

perturbation_function = SineWavePerturbationFunctionHolder(
    amplitude=AMPLITUDE,
    wavelength=WAVELENGTH,
    phase=PHASE,
    axis=AXIS)

configuration = PerturbationConfiguration(
    material=supercell,
    perturbation_function_holder=perturbation_function,
    use_cartesian_coordinates=USE_CARTESIAN_COORDINATES)

if PRESERVE_GEODESIC_DISTANCE:
    builder = CellMatchingDistancePreservingSlabPerturbationBuilder()
else:
    builder = SlabPerturbationBuilder()

### 2.2. Apply perturbation to the material

In [ ]:
from mat3ra.made.tools.build.perturbation import create_perturbation

material_with_perturbation = create_perturbation(configuration, builder)

### 3. Visualize the Result

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_perturbation, "title": f"Material with perturbation"},
           {"material": material_with_perturbation, "title": f"Material with perturbation", "rotation": "-90x"},
           ])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials(material_with_perturbation)